In [1]:
import finnhub
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

# Setup client
finnhub_client = finnhub.Client(api_key=os.environ['FINNHUB_KEY'])

is_custom_date = os.getenv("CUSTOM_DATE", 'False') == 'True'

False

In [2]:
import requests
import bs4 as bs

def get_tickers():
    """Method that gets the stock symbols from companies listed in the S&P 500

    Return
    ------
    `tickers` : list
        S&P 500 company symbols
    """
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find_all('table')[0]  # Grab the first table

    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.strip('\n')
        tickers.append(ticker)

    return tickers

In [3]:
tickers = get_tickers()
# tickers

In [4]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

if is_custom_date:
    # Instantiating and checking dates
    start_date = "2024-09-22"
    end_date = "2024-02-22"
    
    start_date_ = datetime.strptime(start_date, "%Y-%m-%d")  # datetime object
    end_date_ = datetime.strptime(end_date, "%Y-%m-%d")  # datetime object
    delta_date = abs((end_date_ - start_date_).days)  # number of days between 2 dates
    
    if start_date_ > end_date_:
        raise Exception("'start_date' is after 'end_date'")
    
    t = (datetime.now() - relativedelta(years=1))
    
    if start_date_ <= t:
        raise Exception("'start_date' is older than 1 year. It doesn't work with the free version of FinnHub")
else:
    # Default is to instantiate to 1 year prior
    start_date = (datetime.now() - relativedelta(years=1)).strftime("%Y-%m-%d")
    end_date = datetime.now().strftime("%Y-%m-%d")

In [5]:
print(start_date)
print(end_date)

2023-06-19
2024-06-19


In [6]:
folder_name = 'company_news'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
for ticker in tickers:
    # Pull all company news based on ticker list
    # Need to use _from instead of from to avoid conflict
    data = finnhub_client.company_news(ticker, _from=start_date, to=end_date)
    df = pd.DataFrame.from_dict(data, orient='columns')
    df.to_csv(f'{folder_name}/{ticker}.csv')

FinnhubAPIException: FinnhubAPIException(status_code: 429): API limit reached. Please try again later. Remaining Limit: 0